In [1]:
#此段下載資料，如果沒有要下載勿重複使用
"""
def real_estate_crawler(year, season):
  if year > 1000:
    year -= 1911

  # download real estate zip file
  res = requests.get("https://plvr.land.moi.gov.tw//DownloadSeason?season="+str(year)+"S"+str(season)+"&type=zip&fileName=lvr_landcsv.zip")

  # save content to file
  fname = str(year)+str(season)+'.zip'
  open(fname, 'wb').write(res.content)

  # make additional folder for files to extract
  folder = 'real_estate' + str(year) + str(season)
  if not os.path.isdir(folder):
    os.mkdir(folder)

  # extract files to the folder
  with zipfile.ZipFile(fname, 'r') as zip_ref:
    zip_ref.extractall(folder)

  time.sleep(10)

real_estate_crawler(101, 3)
real_estate_crawler(101, 4)

for year in range(102, 108):
  for season in range(1,5):
    print(year, season)
    real_estate_crawler(year, season)

real_estate_crawler(108, 1)
real_estate_crawler(108, 2)
"""

'\ndef real_estate_crawler(year, season):\n  if year > 1000:\n    year -= 1911\n\n  # download real estate zip file\n  res = requests.get("https://plvr.land.moi.gov.tw//DownloadSeason?season="+str(year)+"S"+str(season)+"&type=zip&fileName=lvr_landcsv.zip")\n\n  # save content to file\n  fname = str(year)+str(season)+\'.zip\'\n  open(fname, \'wb\').write(res.content)\n\n  # make additional folder for files to extract\n  folder = \'real_estate\' + str(year) + str(season)\n  if not os.path.isdir(folder):\n    os.mkdir(folder)\n\n  # extract files to the folder\n  with zipfile.ZipFile(fname, \'r\') as zip_ref:\n    zip_ref.extractall(folder)\n\n  time.sleep(10)\n\nreal_estate_crawler(101, 3)\nreal_estate_crawler(101, 4)\n\nfor year in range(102, 108):\n  for season in range(1,5):\n    print(year, season)\n    real_estate_crawler(year, season)\n\nreal_estate_crawler(108, 1)\nreal_estate_crawler(108, 2)\n'

In [2]:
#爬取了實價登入所有歷史數據
import requests
import os
import zipfile
import time
from datetime import datetime

In [3]:
"""'公寓(5樓含以下無電梯)', '透天厝', '套房(1房1廳1衛)', '住宅大樓(11層含以上有電梯)',
       '華廈(10層含以下有電梯)',"""

"'公寓(5樓含以下無電梯)', '透天厝', '套房(1房1廳1衛)', '住宅大樓(11層含以上有電梯)',\n       '華廈(10層含以下有電梯)',"

In [4]:
"""x_lvr_land_a：房屋買賣交易
x_lvr_land_b：新成屋交易
x_lvr_land_c：租房交易
"""

'x_lvr_land_a：房屋買賣交易\nx_lvr_land_b：新成屋交易\nx_lvr_land_c：租房交易\n'

In [60]:
#本案例只對二手房市做預測
import os
import pandas as pd
import numpy as np
import math
dirs = [d for d in os.listdir() if d[:4] == 'real' and int(d[11:14]) >=105 ]

dfs = []

for d in dirs:
    print(d)

    df = pd.read_csv(os.path.join(d,'a_lvr_land_a.csv'), index_col=False)
    df['Q'] = d[-1]
    dfs.append(df.iloc[1:])
    
df = pd.concat(dfs, sort=True)

real_estate1051
real_estate1052
real_estate1053
real_estate1054
real_estate1061
real_estate1062
real_estate1063
real_estate1064
real_estate1071
real_estate1072
real_estate1073
real_estate1074
real_estate1081
real_estate1082


In [61]:
#資訊還必須再經過處理，才會讓我們待會的資料分析更好下手

In [62]:
temp=[0,1,2,3,4,5,6,7,8,9]
temp[-4:-2]

[6, 7]

In [63]:
def process(df):
  df['year'] = df['交易年月日'].str[:-4].astype(int) + 1911
  df['month'] = df['交易年月日'].str[-4:-2].astype(int)
  df['day'] = df['交易年月日'].str[-2:].astype(int)
  # 不同名稱同項目資料合併
  df['單價元/平方公尺'].fillna(df['單價元/平方公尺'], inplace=True)
  df.drop(columns='單價元/平方公尺')
  # 平方公尺換成坪
  df['單價元/平方公尺'] = df['單價元/平方公尺'].astype(float)
  df['建物型態2'] = df['建物型態'].str.split('(').str[0]
  # 刪除有備註之交易（多為親友交易、價格不正常之交易）
  df = df[df['備註'].isnull()]

  df.index = pd.to_datetime((df['交易年月日'].str[:-4].astype(int) + 1911).astype(str) + df['交易年月日'].str[-4:] ,errors='coerce')


process(df)

In [64]:
"""
set:集合
集合（set）是一个无序的不重复元素序列。

可以使用大括号 { } 或者 set() 函数创建集合，
注意：创建一个空集合必须用 set() 而不是 { }，因为 { } 是用来创建一个空字典。
"""

'\nset:集合\n集合（set）是一个无序的不重复元素序列。\n\n可以使用大括号 { } 或者 set() 函数创建集合，\n注意：创建一个空集合必须用 set() 而不是 { }，因为 { } 是用来创建一个空字典。\n'

In [65]:
#篩選建物型態
df=df[df["建物型態"].isin(['公寓(5樓含以下無電梯)',  '套房(1房1廳1衛)', '住宅大樓(11層含以上有電梯)','華廈(10層含以下有電梯)'])]

In [66]:
#check
df["建物型態"].unique()

array(['公寓(5樓含以下無電梯)', '套房(1房1廳1衛)', '住宅大樓(11層含以上有電梯)', '華廈(10層含以下有電梯)'],
      dtype=object)

In [67]:
set(df['鄉鎮市區'])

{'中山區',
 '中正區',
 '信義區',
 '內湖區',
 '北投區',
 '南港區',
 '士林區',
 '大同區',
 '大安區',
 '文山區',
 '松山區',
 '萬華區'}

In [68]:
df.columns

Index(['Q', '主要建材', '主要用途', '交易年月日', '交易標的', '交易筆棟數', '備註', '單價元/平方公尺',
       '單價元平方公尺', '土地區段位置/建物區段門牌', '土地區段位置建物區段門牌', '土地移轉總面積平方公尺', '建物型態',
       '建物現況格局-廳', '建物現況格局-房', '建物現況格局-衛', '建物現況格局-隔間', '建物移轉總面積平方公尺',
       '建築完成年月', '有無管理組織', '移轉層次', '編號', '總價元', '總樓層數', '車位移轉總面積平方公尺', '車位總價元',
       '車位類別', '都市土地使用分區', '鄉鎮市區', '非都市土地使用分區', '非都市土地使用編定', 'year', 'month',
       'day', '建物型態2'],
      dtype='object')

In [69]:
df.主要建材.unique()

array(['鋼筋混凝土造', '加強磚造', '見其他登記事項', '鋼骨鋼筋混凝土造', '磚造', '見使用執照', nan,
       '鋼骨混凝土造', '預力混凝土造', '壁式預鑄鋼筋混凝土造', '鋼筋混凝土加強磚造', '土磚石混合造', '木造',
       '鋼造'], dtype=object)

In [70]:
try:
    special_material = ['見其他登記事項','見使用執照', nan]
except:
    special_material = []

In [71]:
#可以發現nan無法直接進入list，所以isin沒辦法處理nan
special_material

[]

In [72]:
#nan先轉換成'NaN'後再用isin刪除
na_equiv = 'NaN'
df["主要建材"].replace(pd.np.nan, na_equiv,inplace=True)
try:
    specialcategory = ['見其他登記事項','見使用執照', 'NaN']
except:
    specialcategory = []
    
def deal_special_dump(df,col,specialcategory):
    specialcategory.append('NaN')
    df[col].replace(pd.np.nan,'NaN',inplace=True)
    df=df[~df[col].isin(specialcategory)]
    print("unique col now:",df[col].unique())
    return df

def deal_special_dump_nan0(df,col,specialcategory=[]):
    if df[col].iloc[0].isdigit() is True:
        df[col].replace(pd.np.nan,0,inplace=True)
    else:
        df[col].replace(pd.np.nan,'NaN',inplace=True)
    df=df[~df[col].isin(specialcategory)]
    print("unique col now:",df[col].unique())
    return df

def deal_positive_cat(df,col,specialcategory):
    df=df[df[col].isin(specialcategory)]
    print("unique col now:",df[col].unique())
    return df

def deal_positive_contain(df,col,specialcategory):
    df[col].replace(pd.np.nan,'NaN',inplace=True)
    df=df[df[col].str.contains('|'.join(specialcategory))]
    return df

In [73]:
df.主要建材.unique()

array(['鋼筋混凝土造', '加強磚造', '見其他登記事項', '鋼骨鋼筋混凝土造', '磚造', '見使用執照', 'NaN',
       '鋼骨混凝土造', '預力混凝土造', '壁式預鑄鋼筋混凝土造', '鋼筋混凝土加強磚造', '土磚石混合造', '木造',
       '鋼造'], dtype=object)

In [74]:
df=deal_special_dump(df,"主要建材",['見使用執照', '見其他登記事項'])

unique col now: ['鋼筋混凝土造' '加強磚造' '鋼骨鋼筋混凝土造' '磚造' '鋼骨混凝土造' '預力混凝土造' '壁式預鑄鋼筋混凝土造'
 '鋼筋混凝土加強磚造' '土磚石混合造' '木造' '鋼造']


In [75]:
df.主要用途.unique()

array(['住家用', '商業用', '見使用執照', '見其他登記事項', nan, '住商用', '工業用', '住工用', '停車空間',
       '工商用', '國民住宅'], dtype=object)

In [76]:
df['主要用途'].iloc[0]

'住家用'

In [77]:
df=deal_special_dump(df,"主要用途",['見使用執照', '見其他登記事項'])

unique col now: ['住家用' '商業用' '住商用' '工業用' '住工用' '停車空間' '工商用' '國民住宅']


In [78]:
df.交易標的.unique()

array(['房地(土地+建物)', '房地(土地+建物)+車位', '建物'], dtype=object)

In [79]:
df.交易年月日

1       1041005
33      1041003
34      1040921
35      1041007
36      1040925
37      1041012
38      1040902
40      1041020
41      1040916
42      1041003
43      1040927
44      1040823
45      1041006
46      1040918
48      1041020
49      1041018
50      1040827
53      1041019
54      1041016
55      1041012
56      1041008
61      1040922
62      1041006
63      1000224
64      1040912
65      1041020
67      1041014
68      1041008
69      1020430
70      1041013
         ...   
5528    1080501
5530    1080406
5531    1080401
5532    1080225
5533    1071207
5534    1080401
5535    1080323
5536    1080405
5537    1071029
5538    1080312
5541    1080412
5544    1080408
5545    1080326
5546    1080413
5547    1080331
5548    1080418
5550    1080408
5551    1080227
5555    1080429
5556    1080422
5557    1080328
5558    1080328
5559    1080115
5560    1080410
5562    1060422
5563    1080410
5565    1080330
5568    1080218
5569    1080422
5570    1080403
Name: 交易年月日, Length: 466

In [80]:
df.year

1       2015
33      2015
34      2015
35      2015
36      2015
37      2015
38      2015
40      2015
41      2015
42      2015
43      2015
44      2015
45      2015
46      2015
48      2015
49      2015
50      2015
53      2015
54      2015
55      2015
56      2015
61      2015
62      2015
63      2011
64      2015
65      2015
67      2015
68      2015
69      2013
70      2015
        ... 
5528    2019
5530    2019
5531    2019
5532    2019
5533    2018
5534    2019
5535    2019
5536    2019
5537    2018
5538    2019
5541    2019
5544    2019
5545    2019
5546    2019
5547    2019
5548    2019
5550    2019
5551    2019
5555    2019
5556    2019
5557    2019
5558    2019
5559    2019
5560    2019
5562    2017
5563    2019
5565    2019
5568    2019
5569    2019
5570    2019
Name: year, Length: 46636, dtype: int32

In [81]:
df = deal_positive_cat(df,"year",[2015,2016,2017,2018,2019])


unique col now: [2015 2016 2017 2018 2019]


In [82]:
#要修改
df.交易標的.unique()

array(['房地(土地+建物)', '房地(土地+建物)+車位', '建物'], dtype=object)

In [83]:
df.交易筆棟數.unique()

array(['土地1建物1車位0', '土地2建物1車位0', '土地1建物1車位1', '土地5建物1車位0', '土地1建物1車位2',
       '土地6建物1車位0', '土地3建物1車位1', '土地1建物2車位1', '土地2建物1車位1', '土地3建物1車位0',
       '土地5建物2車位1', '土地2建物1車位2', '土地4建物1車位0', '土地1建物1車位4', '土地1建物2車位0',
       '土地7建物1車位0', '土地0建物1車位1', '土地2建物2車位1', '土地0建物1車位0', '土地1建物2車位2',
       '土地8建物1車位0', '土地3建物3車位0', '土地1建物1車位3', '土地2建物2車位2', '土地2建物2車位0',
       '土地11建物1車位0', '土地2建物1車位4', '土地0建物1車位3', '土地3建物1車位2', '土地1建物3車位0',
       '土地2建物3車位1', '土地4建物1車位1', '土地1建物6車位4', '土地4建物1車位2', '土地0建物2車位0',
       '土地5建物1車位1', '土地2建物1車位3', '土地1建物1車位6', '土地1建物11車位0', '土地1建物5車位0',
       '土地1建物4車位0', '土地1建物12車位0', '土地4建物2車位0', '土地0建物1車位4', '土地10建物1車位0',
       '土地16建物1車位1', '土地3建物2車位0', '土地2建物42車位0', '土地2建物12車位0', '土地2建物1車位5',
       '土地2建物3車位0', '土地0建物1車位2', '土地3建物1車位3', '土地2建物3車位2', '土地1建物1車位5',
       '土地13建物1車位0', '土地1建物2車位4', '土地2建物4車位0', '土地4建物2車位1', '土地0建物2車位2',
       '土地1建物3車位3', '土地2建物2車位3', '土地0建物3車位0', '土地1建物2車位6', '土地1建物2車位5',
       '土地2建物5車位3', '土地1建物3車位4', '土地12建物1車位0', '土地0建物4車位

In [84]:
df = df[df['備註'].isnull()]

In [85]:
df['土地區段位置/建物區段門牌'][100:200]

356         臺北市內湖區陽光街161巷1弄1~30號
358          臺北市內湖區成功路二段361~390號
361         臺北市中山區南京東路一段13巷1~30號
362           臺北市中山區建國北路一段31~60號
366               臺北市中正區泰安街1~30號
370            臺北市大同區重慶北路一段1~30號
373       臺北市文山區興隆路四段58巷26弄1~30號
375            臺北市文山區興德路20巷1~30號
388            臺北市中正區?門街121~150號
390         臺北市文山區木新路三段95巷31~60號
392        臺北市萬華區和平西路三段298巷1~30號
403    臺北市內湖區民權東路六段280巷69弄61~90號
404        臺北市中山區新生北路二段60巷31~60號
410         臺北市中山區中山北路二段16巷1~30號
411     臺北市內湖區民權東路六段180巷121~150號
413               臺北市內湖區金湖路1~30號
415           臺北市南港區福德街310巷1~30號
416         臺北市信義區和平東路三段211~240號
419            臺北市松山區延吉街46巷1~30號
420             臺北市信義區松隆路9巷1~30號
426        臺北市士林區德行東路74巷15弄1~30號
427              臺北市士林區菁山路61~90號
428              臺北市士林區菁山路61~90號
429          臺北市北投區大度路三段271~300號
430          臺北市北投區石牌路一段58巷1~30號
433        臺北市士林區延平北路六段392巷1~30號
435          臺北市北投區東華街一段361~390號
436          臺北市北投區東華街一段361~390號
437          臺北市北投區東華街一段361~390號
438              臺北市北投區中山路31~60號
          

In [86]:
df.土地移轉總面積平方公尺.unique()

array(['25.44', '39.2', '12.4', ..., '62.19', '16.68', '99.47'],
      dtype=object)

In [87]:
df.建物型態.unique()

array(['公寓(5樓含以下無電梯)', '華廈(10層含以下有電梯)', '住宅大樓(11層含以上有電梯)', '套房(1房1廳1衛)'],
      dtype=object)

In [88]:
df['建物現況格局-廳'].unique()

array(['2', '1', '0', '3', '6', '4', '9', '80', '24', '5', '7', '8', '10'],
      dtype=object)

In [89]:
df['建物現況格局-房'].unique()

array(['3', '2', '4', '0', '1', '5', '6', '7', '9', '140', '39', '8',
       '10', '11', '12', '32', '20'], dtype=object)

In [90]:
df['建物現況格局-衛'].unique()

array(['2', '1', '0', '3', '5', '6', '4', '7', '8', '75', '23', '10', '9',
       '30', '22'], dtype=object)

In [91]:
df.建物移轉總面積平方公尺[0:10]

1                  107.4
36    148.95000000000002
40                 46.08
44                326.63
45                 90.91
46                102.76
49                 99.08
53                172.56
54                151.28
55                104.74
Name: 建物移轉總面積平方公尺, dtype: object

In [92]:
df.建築完成年月[0:10]

1     0701231
36    0830530
40        NaN
44    0981126
45    0710622
46    0690515
49    0690422
53    0660322
54    0690324
55    0691024
Name: 建築完成年月, dtype: object

In [93]:
df = deal_special_dump_nan0(df,'建築完成年月')

unique col now: ['0701231' '0830530' 0 ... '0590217' '0860806' '0900629']


In [157]:

def fill_back_0(strin):
    strin=int(strin)
    #if type(strin)!=str:
    if strin ==0: strin=1
    if int(math.log10(strin))+1==3:
        strin=strin*10000
    elif int(math.log10(strin))+1==5:
        strin=strin*100
    strin=str(strin).zfill(7)
    return strin


In [158]:
df['建築完成年月'] = df.建築完成年月.apply(fill_back_0)

In [159]:
df.columns

Index(['Q', '主要建材', '主要用途', '交易年月日', '交易標的', '交易筆棟數', '備註', '單價元/平方公尺',
       '單價元平方公尺', '土地區段位置/建物區段門牌', '土地區段位置建物區段門牌', '土地移轉總面積平方公尺', '建物型態',
       '建物現況格局-廳', '建物現況格局-房', '建物現況格局-衛', '建物現況格局-隔間', '建物移轉總面積平方公尺',
       '建築完成年月', '有無管理組織', '移轉層次', '編號', '總價元', '總樓層數', '車位移轉總面積平方公尺', '車位總價元',
       '車位類別', '都市土地使用分區', '鄉鎮市區', '非都市土地使用分區', '非都市土地使用編定', 'year', 'month',
       'day', '建物型態2', '建築完成年月2', 'b_year', 'b_month', 'b_day', 'b_time'],
      dtype='object')

In [160]:
df['建築完成年月'].str[0:4]

1       0701
36      0830
40      0000
44      0981
45      0710
46      0690
49      0690
53      0660
54      0690
55      0691
56      0971
65      0690
67      0650
70      0000
71      0751
75      0770
85      1020
86      0860
88      0740
91      0000
97      0671
102     0700
104     0640
109     0700
112     1011
113     0841
114     0751
117     0950
118     0970
121     0840
        ... 
5447    0681
5448    0570
5456    0850
5458    1061
5460    0860
5464    1011
5466    0630
5468    0990
5475    0690
5476    0000
5478    0710
5479    0890
5483    0970
5498    0970
5501    0590
5515    0690
5517    0670
5523    0780
5525    0611
5528    0610
5535    0730
5537    0590
5541    0730
5545    0860
5546    0830
5550    0850
5555    0900
5560    0610
5563    1000
5565    0991
Name: 建築完成年月, Length: 21879, dtype: object

In [173]:
df['b_year'] = (df['建築完成年月'].str[:3].fillna(0).astype(int) + 1911).astype(str).str.zfill(4)

df['b_month'] = df['建築完成年月'].str[3:5].fillna(1).astype(str).str.zfill(2)
df['b_month'].replace([0,"00","0"],"01", inplace=True)
df['b_day'] = df['建築完成年月'].str[5:].fillna(1).astype(str).str.zfill(2)
df['b_day'].replace([0,"00","0"],"01", inplace=True)

In [174]:
df['b_day'][0:20]

1     31
36    30
40    01
44    26
45    22
46    15
49    22
53    22
54    24
55    24
56    24
65    28
67    03
70    01
71    17
75    23
85    03
86    30
88    22
91    01
Name: b_day, dtype: object

In [175]:
df[df.index.duplicated()]

,Q,主要建材,主要用途,交易年月日,交易標的,交易筆棟數,備註,單價元/平方公尺,單價元平方公尺,土地區段位置/建物區段門牌,...,非都市土地使用編定,year,month,day,建物型態2,建築完成年月2,b_year,b_month,b_day,b_time
1,2,鋼筋混凝土造,住家用,1050115,房地(土地+建物)+車位,土地1建物1車位1,NaN,151523.0,NaN,臺北市文山區羅斯福路五段236巷3弄31~60號,...,NaN,2016,1,15,住宅大樓,0981224,2009,12,24,12-24-2009
36,2,鋼筋混凝土造,住家用,1050114,房地(土地+建物),土地1建物1車位0,NaN,135685.0,NaN,臺北市北投區承德路七段342巷1~30號,...,NaN,2016,1,14,公寓,0660214,1977,02,14,02-14-1977
40,2,鋼筋混凝土造,住家用,1050104,房地(土地+建物)+車位,土地3建物6車位1,NaN,126299.0,NaN,臺北市北投區公館路167巷1~30號,...,NaN,2016,1,4,住宅大樓,0851104,1996,11,04,11-04-1996
44,2,鋼筋混凝土造,住家用,1050101,房地(土地+建物)+車位,土地3建物1車位1,NaN,108740.0,NaN,臺北市文山區興隆路二段203巷121~150號,...,NaN,2016,1,1,住宅大樓,1030605,2014,06,05,06-05-2014
46,2,鋼筋混凝土造,商業用,1050113,房地(土地+建物)+車位,土地1建物1車位1,NaN,137273.0,NaN,臺北市內湖區民權東路六段151~180號,...,NaN,2016,1,13,華廈,0960808,2007,08,08,08-08-2007
49,2,鋼筋混凝土造,商業用,1041224,房地(土地+建物),土地1建物1車位0,NaN,260015.0,NaN,臺北市信義區光復南路511~540號,...,NaN,2015,12,24,住宅大樓,0891124,2000,11,24,11-24-2000
54,2,鋼筋混凝土造,住家用,1041229,房地(土地+建物),土地2建物1車位0,NaN,249707.0,NaN,臺北市大安區新生南路二段30巷1~30號,...,NaN,2015,12,29,公寓,0000001,1911,01,01,00-01-1911
65,2,鋼筋混凝土造,住家用,1050107,房地(土地+建物),土地1建物1車位0,NaN,251042.0,NaN,臺北市大安區新生南路一段103巷31~60號,...,NaN,2016,1,7,華廈,0000001,1911,01,01,00-01-1911
75,2,鋼筋混凝土造,住家用,1041227,房地(土地+建物),土地1建物1車位0,NaN,156264.0,NaN,臺北市士林區士商路61~90號,...,NaN,2015,12,27,公寓,0700226,1981,02,26,02-26-1981
117,2,鋼筋混凝土造,住家用,1041231,房地(土地+建物),土地1建物1車位0,NaN,199203.0,NaN,臺北市中山區農安街31~60號,...,NaN,2015,12,31,套房,0680420,1979,04,20,04-20-1979


In [176]:
df[['建築完成年月','b_year','b_month','b_day']]

,建築完成年月,b_year,b_month,b_day
1,0701231,1981,12,31
36,0830530,1994,05,30
40,0000001,1911,01,01
44,0981126,2009,11,26
45,0710622,1982,06,22
46,0690515,1980,05,15
49,0690422,1980,04,22
53,0660322,1977,03,22
54,0690324,1980,03,24
55,0691024,1980,10,24


In [177]:
df['b_time'] =  df.b_month+"-"+df.b_day+"-"+df.b_year

In [178]:
df['b_time'][0:50]

1      12-31-1981
36     05-30-1994
40     01-01-1911
44     11-26-2009
45     06-22-1982
46     05-15-1980
49     04-22-1980
53     03-22-1977
54     03-24-1980
55     10-24-1980
56     12-24-2008
65     04-28-1980
67     06-03-1976
70     01-01-1911
71     10-17-1986
75     04-23-1988
85     04-03-2013
86     01-30-1997
88     05-22-1985
91     01-01-1911
97     12-01-1978
102    03-18-1981
104    02-22-1975
109    03-31-1981
112    12-26-2012
113    12-04-1995
114    11-06-1986
117    02-09-2006
118    01-10-2008
121    04-11-1995
123    12-27-2004
124    06-30-1970
125    01-01-1911
128    10-21-1981
129    07-31-2007
130    05-20-2015
131    05-20-2015
138    12-09-1983
139    11-11-2005
148    01-01-1911
149    02-02-2005
155    06-28-2000
156    12-24-1987
160    03-01-1977
161    03-12-2008
162    05-20-1985
163    10-23-1986
164    05-22-2015
165    02-10-2015
169    11-20-1985
Name: b_time, dtype: object

In [156]:
df['b_time22'] =pd.to_datetime(df.b_time,format='%m-%d-%Y')

ValueError: time data '00-01-1911' does not match format '%m-%d-%Y' (match)

In [155]:
#df['b_time22']=pd.to_datetime(dict(year=df.b_year, month=df.b_month, day=df.b_day))

In [164]:
df['b_time22'] 

1       12-31-1981
36      05-30-1994
40      00-00-1911
44      11-26-2009
45      06-22-1982
46      05-15-1980
49      04-22-1980
53      03-22-1977
54      03-24-1980
55      10-24-1980
56      12-24-2008
65      04-28-1980
67      06-03-1976
70      00-00-1911
71      10-17-1986
75      04-23-1988
85      04-03-2013
86      01-30-1997
88      05-22-1985
91      00-00-1911
97      12-01-1978
102     03-18-1981
104     02-22-1975
109     03-31-1981
112     12-26-2012
113     12-04-1995
114     11-06-1986
117     02-09-2006
118     01-10-2008
121     04-11-1995
           ...    
5447    11-03-1979
5448    03-25-1968
5456    06-24-1996
5458    12-22-2017
5460    06-03-1997
5464    11-12-2012
5466    09-26-1974
5468    09-03-2010
5475    03-21-1980
5476    00-00-1911
5478    02-27-1982
5479    06-28-2000
5483    05-02-2008
5498    03-12-2008
5501    02-17-1970
5515    06-06-1980
5517    09-18-1978
5523    05-16-1989
5525    10-06-1972
5528    08-05-1972
5535    04-16-1984
5537    09-2

In [150]:
b_t = ['Y'] + '-' + i['M'] + '-' + i['D'] + ' ' + i['h'] + ':' + i['m'] + ':' + i['s'] 
pd.to_datetime(j)
考慮用字串

TypeError: can only concatenate list (not "str") to list

In [90]:
df['有無管理組織'].unique()

array(['無', '有'], dtype=object)

In [ ]:
df["總樓層數"]=df["總樓層數"].astype(str)

In [ ]:
#before give_index we have to remove rows we don't want
df.index = range(df.shape[0])

In [ ]:
#先轉總樓層數!

In [ ]:
#-*- coding: cp936 -*-
import re
import string

digit = {'ㄧ': 1,'一': 1 ,'壹': 1, '一': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9}

def _chn_to_numeric(s):
    num = 0
    if s:
        if s != s:
            s="null"
        idx_q, idx_b, idx_s, idx_f = s.find('千'), s.find('百'), s.find('十'), s.find('層')
        if idx_q != -1:
            num += digit[s[idx_q - 1:idx_q]] * 1000
        if idx_b != -1:
            num += digit[s[idx_b - 1:idx_b]] * 100
        if idx_s != -1:
            # 十前忽略一的处理
            #get 如果第一個沒有值就回傳第2個
            num += digit.get(s[idx_s - 1:idx_s], 1) * 10
        if idx_f != -1:
            # 十前忽略一的处理
            if s[idx_f - 1:idx_f]!="十":
                num += digit.get(s[idx_f - 1:idx_f], 1)
        if num == 0:
            num = 1
        #if s[-1] in digit:
        #    num += digit[s[-1]]
    return num

def trans(chn):
    chn=str(chn)
    chn = chn.replace('零', '')
    idx_y, idx_w = chn.rfind('億'), chn.rfind('萬')
    if idx_w < idx_y:
        idx_w = -1
    num_y, num_w = 100000000, 10000
    if idx_y != -1 and idx_w != -1:
        return trans(chn[:idx_y]) * num_y + _chn_to_numeric(chn[idx_y + 1:idx_w]) * num_w + _chn_to_numeric(chn[idx_w + 1:])
    elif idx_y != -1:
        return trans(chn[:idx_y]) * num_y + _chn_to_numeric(chn[idx_y + 1:])
    elif idx_w != -1:
        return _chn_to_numeric(chn[:idx_w]) * num_w + _chn_to_numeric(chn[idx_w + 1:])
    return _chn_to_numeric(chn)

In [ ]:
df["總樓層數2"]=df["總樓層數"].apply(trans)

In [ ]:
df[["總樓層數2","總樓層數"]][50:70]

In [ ]:
#to there

In [ ]:
def foo(t_floor,all_floor):
    if t_floor=="全":
        return range(1,all_floor+1)
    else:
        return t_floor

df['移轉層次_2'] = df.apply(lambda x: foo(x["移轉層次"], x["總樓層數"]), axis=1)

In [ ]:
df['移轉層次_2'][0:50]

In [ ]:
def foo(t_floor,all_floor):
    if t_floor=="全":
        return range(1,all_floor+1)
    else:
        return t_floor

df['移轉層次_2'] = df.apply(lambda x: foo(x["移轉層次"], x["總樓層數"]), axis=1)

In [ ]:
df['移轉層次_2'][0:50]

In [ ]:
#-*- coding: cp936 -*-
import re
import string
import sys

digit = {'ㄧ': 1,'一': 1 ,'壹': 1, '一': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9}

"""要修改 沒辦法回傳多樓層"""
def chn_to_numeric_floor(chn):
    deli = "層"
    chn_list=[]
    if chn != chn:
        chn="null"
    if isinstance(chn,str):
        #for line in chn:
        str_s =  [e+deli for e in chn.split(deli) if e]
        for chn in str_s:    
            idx_y, idx_w = chn.rfind('億'), chn.rfind('萬')
            if idx_w < idx_y:
                idx_w = -1
            num_y, num_w = 100000000, 10000
            if idx_y != -1 and idx_w != -1:
                chn_list.append(chn_to_numeric_floor(chn[:idx_y]) * num_y + _chn_to_numeric(chn[idx_y + 1:idx_w]) * num_w + _chn_to_numeric(chn[idx_w + 1:]))
            elif idx_y != -1:
                chn_list.append(chn_to_numeric_floor(chn[:idx_y]) * num_y + _chn_to_numeric(chn[idx_y + 1:]))
            elif idx_w != -1:
                chn_list.append(_chn_to_numeric(chn[:idx_w]) * num_w + _chn_to_numeric(chn[idx_w + 1:]))
            else:
                chn_list.append(_chn_to_numeric(chn))
        return chn_list
    else:
        return chn



df['移轉層次_2'] = df['移轉層次_2'].apply(chn_to_numeric_floor)

In [ ]:
df[['移轉層次','移轉層次_2']][0:50]

In [ ]:
print(trans('五層')) 
print(trans('一層')) 
print(trans('十') == 10) 
print(trans('一百零一') == 101)
print(trans('九百二十一') == 921)
print(trans('五十六万零一十') == 560010)
print(trans('一万亿零二千一百零一') == 1000000002101)
print(trans('一万亿二千一百万零一百零一') == 1000021000101)
print(trans('一万零二百三十亿四千零七千八百九十') == 1023000007890)

In [ ]:
lis= []
lis.append(2)
lis.append(99)
lis


In [ ]:
df

In [ ]:
#圖表分析
prices = {}
for district in set(df['鄉鎮市區']):
    cond = (
        (df['主要用途'] == '住家用')
        & (df['鄉鎮市區'] == district)
        & (df['單價元平方公尺'] < df["單價元平方公尺"].quantile(0.9))
        & (df['單價元平方公尺'] > df["單價元平方公尺"].quantile(0.1))
        )
    
    groups = df[cond]['year']
    
    prices[district] = df[cond]['單價元平方公尺'].astype(float).groupby(groups).mean().loc[2012:]

prices = pd.DataFrame(prices)
prices.head()

In [ ]:
prices.plot()

In [ ]:
prices.mean(axis=1).plot()

In [ ]:
building_type_prices = {}
for building_type in set(df['建物型態2']):
    cond = (
        (df['主要用途'] == '住家用')
        & (df['單價元平方公尺'] < df["單價元平方公尺"].quantile(0.8))
        & (df['單價元平方公尺'] > df["單價元平方公尺"].quantile(0.2))
        & (df['建物型態2'] == building_type)
        )
    
    building_type_prices[building_type] = df[cond]['單價元平方公尺'].groupby(df[cond]['year']).mean().loc[2012:]
pd.DataFrame(building_type_prices)[['公寓', '住宅大樓', '套房', '華廈']].plot()

In [ ]:
for district in set(df['鄉鎮市區']):
  dfdistrict = df[df['鄉鎮市區'] == district]
  dfdistrict['單價元平方公尺'][dfdistrict['單價元平方公尺'] < 1000000].hist(bins=100, alpha=0.7)

plt.xlim(0, 1000000)

In [ ]:
#以上參考https://www.finlab.tw/